# Ephemeris Data 

Overview of handling ephemeris data in CASA





## Ephemeris Objects 

Since ALMA Cycle 3, the ALMA observatory includes in each raw dataset (SDM) all necessary ephemerides in the so-called Ephemeris table, an XML file which corresponds to the ephemeris used during the observation. Upon import to MS format, the task **importasdm** will translate the xml ephemerides into separate CASA ephemeris tables for each field, which have the same format as those used by the task **setjy**. Examples can be found in the subdirectory \"data/ephemerides/JPL-Horizons\" in each CASA distribution.

The ephemeris tables are automatically attached to the corresponding field(s) and can be used whenever needed. They have two main use cases:

1.  An ephemeris table is necessary for the *spectral* frame transformation of the visibilities to the source rest frame *permanently* when creating a new MS using the tasks **cvel, cvel2,** or **mstransform,** or *on-the-fly* during imaging using the task **tclean** by setting the parameter *specmode* to \"cubesource\". The ephemeris provides the location of the phase center and the radial velocity of the target.
2.  An ephemeris table is also necessary if your observation has tracked some other phase center but you would like to track the location described by the ephemeris. This can be achieved by setting the parameter *phasecenter* of task **tclean** to the string \"TRACKFIELD\". See the **tclean** documentation for more details.

The ephemerides used by ALMA are originally for the observer location of ALMA (topocentric). They use the ICRS coordinate reference frame and typically have a time spacing of ten or twenty minutes. For the later transformation of the spectral reference frame, however, a geocentric ephemeris is necessary. The **importasdm** task will therefore by default also perform a conversion from the ALMA observer location to the geocenter. This is controlled by the **importasdm** parameter *convert_ephem2geo* which is True by default.

The spectral reference frame of the visibilities in the SDM is always topocentric (TOPO), the observer reference frame. To analyze spectral lines in an ephemeris source, it can be helpful to transform the visibilities to the source rest spectral frame (REST). This is in particular necessary when the velocity of the source varies significantly throughout the observation (which can, e.g., happen if the observation is spread over serveral days). As mentioned above, this offline software Doppler-shift correction can be achieved in two ways, either permanently or on-the-fly. This is described further below.

When an ephemeris is attached to a field of the MS FIELD table, the object position does not correspond to the direction columns of the FIELD table but is retrieved by linearly interpolating from the ephemeris table for the given time. The nominal entry of the direction column then *changes its meaning to an angular offset* with respect to the ephemeris. Thus, if the field center is exactly at the position described by the ephemeris, the nominal entries of the direction column are zero for right ascension and declination. The offset feature of the FIELD table direction column comes into play in the case of a mosaic: if there are a number of fields with nearby positions, the fields can share an ephemeris; they all reference the same ephemeris table via the optional column EPHEMERIS_ID, but have different offsets in their direction column entries.

Because the nominal FIELD table direction column entries do not correspond to the actual object position, one should obtain the object position with the following special tool method:

```
msmd.phasecenter()
```

or the more general:

```
ms.getfielddirmeas()
```

(see the inline CASA help for details on these tools). The default time of the position is taken from the TIME column of the FIELD table.


**Permanent spectral frame transformation creating a new MS**

Either with either the task **cvel** or its faster implementation **cvel2** (which uses internally the same code as the task **mstransform**) or with **mstransform**, a new MS can be created which contains visibilities in the spectral reference frame of the source. All three tasks should produce the same result. As a matter of fact, an online Doppler-shift tracking corresponding to the velocity of the source at the beginning of the observation is applied during observations. This online correction allows one to tune the spectral windows adequately to match the requested rest frequencies, but the labelling of the spectral frame of the raw data remains in TOPO.

The user must set the *outframe* parameter of **cvel**, **cvel2**, or **mstransform** to \"SOURCE\". This will lead to a transformation from the TOPO to the GEO reference frame followed by an additional time-dependent Doppler correction according to the radial velocity of the object as given by its ephemeris.

Such MSes should then be imaged using the setting \"cubedata\" for the parameter *specmode* in **tclean**. The resulting spectral reference frame in the MS is named \"REST\".

**On-the-fly spectral frame transformation within tclean**

If a permanent storage of spectrally transformed visibilities in a new MS is not needed, cubes with the spectral frame of the ephemeris object can also be obtained by letting the task **tclean** perform the frame transformation on-the-fly. This is simply achieved by setting parameter *specmode* to the string \"cubesource\". The resulting cube will also have the reference frame named \"REST\".


In summary, with ephemerides included in the ALMA raw data and the added support for this in the **importasdm** task, the user rarely needs to worry about how to obtain the right ephemeris in the right format and how to attach it properly to the MS. This process is transparent and only a few logger messages indicate that it is happening (see CASA Docs pages on [\'Manipulate Ephemeris Objects\'](ephemeris_data.ipynb#Manipulate-Ephemeris-Objects) if a new ephemeris needs to be added to an existing MS). The correct time-dependent positions, radial velocities, and object distances are used in all relevant tasks such as **listobs**, **plotms**, and, as described above, **cvel**, **cvel2**, **mstransform**, and **tclean**. For Solar-System object flux calibrators, the task **setjy** will, however, only extract the nominal position from the SDM ephemeris and otherwise use its internal set of ephemerides since these contain additionally needed object parameters. Care has to be taken when trying to extract the field positions from the FIELD table as the nominal direction column entries will only be offsets (w.r.t. the ephemeris position) when an ephemeris is attached.

<div class="alert alert-warning">
**WARNING**: Virtual model columns in the MS do not correctly support ephemeris objects, although they will run without generating errors or warnings. If any of your calibrators exhibit significant celestial motion on the timescale of your observation (e.g., any solar system object), you must set *usescratch=True* in calls to **setjy**.
</div>

As opposed to ALMA data which use a tabulated representation of the ephemerides, VLA data use a polynomial representation of the positions and radial velocities. Also this representation is supported. The polynomial ephemeris is internally tabulated with a default time step of 0.001 days and then processed as in the ALMA case. The **importasdm** parameter *polyephem_tabtimestep* can be used to control the step size of the tabulation.





## Manipulate Ephemeris Objects 

When an astronomical object has a proper motion, in particular objects in our solar system, a static (RA,Dec) position in the FIELD table of the MeasurementSet will not accurately describe the time-dependent position. Prior to CASA 4.2, there was no support for ephemeris objects other than the built-in reference frames for the Sun, the Moon, and the planets out to PLUTO. With CASA 4.2, several new features were introduced which help the user to attach an arbitrary ephemeris to a given field and work with the object from calibration to imaging. These can be used when no ephemeris table was provided by the observatory, or in cases where the use of an improved ephemeris table is necessary.


**Ephemeris tables**

The CASA format for ephemeris tables was introduced in the early development stages of CASA in connection with the Measures module. The me tool (see [CASA Tools](../api/casatools.rst) on using the tool kit, as well as the inline help on the me tool inside CASA for specific usage) permits position calculations based on ephemerides in this format. Two examples for such tables can be found in the distribution directory in subdirectory data/ephemerides: VGEO is an ephemeris of Venus in the geocentric reference frame while VTOP is an ephemeris for the same object in the TOPO reference fame for the observatory location of the VLA. With the introduction of solar system source models (Butler) in the **setjy** task, a nearly complete set of ephemerides for the larger bodies in our solar system had to be made available. These are stored in nearly the same format as the above examples VGEO and VTOP (but with a few enhancements) in directory data/ephemerides/JPL-Horizons. If your object's ephemeris is among those stored in data/ephemerides/JPL-Horizons, you can simply copy the ephemeris from there. Otherwise, you can request the ephemeris from the JPL-Horizons using the CASA commands (for example):

```
import casatasks.private.request as jplreq

(For CASA 5: import recipes.ephemerides.request as jplreq)

jplreq.request_from_JPL(objnam='Mars',startdate='2012-01-01',enddate='2013-12-31',date_incr='0.1 d', get_axis_orientation=False,
get_axis_ang_orientation=True,
get_sub_long=True, use_apparent=False, get_sep=False,
return_address='YOUR_EMAIL_ADDESS',
mailserver='YOUR_MAIL_SERVER_ADDRESS')
```

where you need to fill in the parameters objnam, startdate, enddate,date_incr (the time interval between individual ephemeris table entries), return_address (your email address where you want to receive the ephemeris), and mailserver (the smtp server through which you want to send the request email). The other parameters should be set as shown. Within a short time, you should receive the requested ephemeris as an email from NASA's JPL-Horizonssystem. Save the email into a file with the "save as" function of your mail client. See the next section on how to attach it to your dataset.


**Using fixplanets to attach ephemerides to a field of a MeasurementSet**

Task importasdm fills the SOURCE coodinates with the correct postions based on the ephemerides table. Alternatively, one can use the task **fixplanets** to set the ephemeris of a given field in a MeasurementSet. Here is an example:

```
fixplanets(vis='uid___A002_X1c6e54_X223.ms',
field='Titan', fixuvw=True, direction='mytitanephemeris')
```

where you need to set the parameters vis to the name of your MS and the parameter field to the name of the field to which you want to attach the ephemeris. The parameter direction must be set to the name of your ephemeris table. Accepted formats are (a) the CASA format (as in VGEO or the ephemerides in data/ephemerides/JPL-Horizons as described above) and (b) the JPL-Horizons mail format which you obtain by saving an ephemeris email you received from JPL-Horizons. The parameter fixuvw should be set to True in order to trigger a recalculation of the UVW coordinates in your MS based on the new ephemeris. The task **fixplanets** can also be used for other field direction modifications. Please refer to the help text of the task.

<div class="alert alert-info">
Note that among the ephemerides in the directory data  /ephemerides/JPL-Horizons/ you should only use those ending in  '_J2000.tab'. They are the ones in J2000 coordinates.
</div>


**Use of the ephemeris after attachment**

Once you have attached the ephemeris to a field of an MS, it will automatically be handled in tasks like **split** and **concat** which need to hand on the ephemeris to their output MSs. In particular **concat** recognizes when fields of the MSs to be concatenated use the same ephemeris and merges these fields if the time coverage of the provided ephemeris in the first MS also covers the observation time of the second MS. The ephemeris of the field in the first MS will then be used for the merged field. In order to inspect the ephemeris attached to a field, you can find it inside the FIELD subdirectory of your MS. The optional column EPHEMERIS_ID in the FIELD table points to the running number of the ephemeris table. A value of −1 indicates that no ephemeris is attached. Note that in case an ephemeris is attached to a field, the direction column entries for that field in the FIELD table will be interpreted as an offset to the ephemeris direction and are therefore set to (0.,0.) by default. This offset feature is used in mosaic observations where several fields share the same ephemeris with different offsets. The time column in the FIELD table should be set to the beginning of the observation for that field and serves as the nominal time for ephemeris queries.


**Spectral frame transformation to the rest frame of the ephemeris object in task cvel**

The ephemerides contain radial velocity information. The task **cvel** can be used to transform spectral windows into the rest frame of the ephemeris object by setting the parameter outframe to "SOURCE" as in the following example:

```
cvel(vis='europa.ms',
outputvis='cvel_europa.ms', outframe='SOURCE', mode = 'velocity',
width = '0.3km/s', restfreq = '354.50547GHz')
```

This will make **cvel** perform a transformation to the GEO reference frame followed by an additional Doppler correction for the radial velocity given by the ephemeris for the each field. (Typically, this should happen after calibration and after splitting out the spectral widows and the target of interest). The result is an MS with a single combined spectral window in reference frame REST. From this frame, further transformations to other reference frames are not possible.


**Ephemerides in ALMA datasets**

The ALMA Science Data Model (the raw data format for ALMA data) now foresees an Ephemeris table. This feature has been in use since the beginning of ALMA Cycle 3 both for science targets and calibrator objects. With ALMA Cycle 3 (or later) datasets, the task **importasdm** will automatically translate the contents of the ASDM Ephemeris table into separate ephemeris tables in CASA format and attach them to the respective fields.

In the case of mosaics, all fields belonging to a mosaic on an ephemeris object will share the same ephemeris. The individual mosaic pointings will use the offset mechanism described above to define the position of each field.


**Imaging ALMA observations with tclean**

The tclean task can automatically handle the imaging of ALMA observations (both single-execution and multi-execution datasets, and both single-field and mosaics) by using the new *phasecenter=\'TRACKFIELD\'* option. This option will use the ephemeris tables attached to each measurementSet by the ALMA control system. These tables will have ultimately been provided by the observatory for the case of large bodies (those selectable in the ALMA Observing Tool), or by the PI as attachments in the Observing Tool for the case of smaller bodies.

<div class="alert alert-warning">
***WARNING***:  if you want to use the old method of concatenating calibrated MeasurementSets by using the *forcesingleephemfield* parameter to create a common joint ephemeris table, then you must still set *phasecenter='TRACKFIELD'* in tclean order to get a sensible image, even though you are passing it only one (concatenated) MeasurementSet. If not, you may get a corrupt image, even if you select a subset of data only from the first execution block in the concatenated MS.
</div>


**Imaging observations from other telescopes with tclean**

For non-ALMA data, or to use a newer ephemeris than was available at the time of the ALMA observations, the user may set the phasecenter parameter to the name of an ephemeris file, constructed as described in the earlier section above. Alternatively, the user may set the phasecenter to the common name of the following bodies: \'MERCURY\', \'VENUS\', \'MARS\', \'JUPITER\', \'SATURN\', \'URANUS\', \'NEPTUNE\', \'PLUTO\', \'SUN\', \'MOON\', in which case the standard DE200 ephemeris table distributed with CASA will be used.
